In [365]:
import pandas as pd
import plotly.graph_objects as go

# Rewards

In [366]:
data =pd.read_csv('logs/reward.csv', header=None)
data.columns = ['iteration', 'value']
chunk_size = 150
batch_size = 32
data =data.groupby(lambda idx: (idx+1) // chunk_size).agg({'iteration': ['max'], 'value': 'sum'})
data.columns = data.columns.droplevel(1)
data['value'] /= (chunk_size * batch_size)

In [367]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Mean Reward')
fig.update_layout(title_text="Reward")
fig.add_trace(go.Scatter(x=data['iteration'][:-1], y=data['value'][:-1],
                    mode='lines+markers', name='markers', marker=dict(
            color='MediumPurple',
            opacity=0.1,
        )))

# Gradient Flow

In [368]:
data =pd.read_csv('logs/grad_flow.csv', header=None)

In [369]:
data[2] = data[2].str.replace('tensor\(', '').str.replace(", device='cuda:0'\)", '').astype('float64')
data[3] = data[3].str.replace('tensor\(', '').str.replace(", device='cuda:0'\)", '').astype('float64')

In [370]:
data.columns = ['iteration', 'layer', 'mean_grad', 'max_grad']

In [371]:
data[['layer', 'mean_grad', 'max_grad']].groupby('layer').agg({'mean_grad':['max', 'mean', 'median'], 'max_grad':['max', 'mean', 'median']})

mean_grad  \
                                                             max   
layer                                                              
decoder.attn.linear_in.weight                       1.920700e-07   
decoder.attn.linear_out.weight                      3.667400e-06   
decoder.rnn.layers.0.weight_hh                      1.465900e-07   
decoder.rnn.layers.0.weight_ih                      7.884000e-07   
decoder_embeddings.make_embedding.emb_luts.0.we...  6.099500e-07   
encoder.rnn.weight_hh_l0                            1.675300e-07   
encoder.rnn.weight_hh_l0_reverse                    1.385500e-07   
encoder.rnn.weight_ih_l0                            2.627900e-06   
encoder.rnn.weight_ih_l0_reverse                    2.758300e-06   
encoder_embeddings.make_embedding.emb_luts.0.we...  9.842300e-07   
generator.advantages.0.weight_mu                    7.779700e-06   
generator.advantages.0.weight_sigma                 9.324000e-06   
generator.advantages.2.weight_mu                    9.571500e-06   
generator.advantages.2.weight_sigma                 5.387200e-06   
generator.value.0.weight_mu                         3.803500e-06   
generator.value.0.weight_sigma                      2.590000e-06   
generator.value.2.weight_mu                         3.000000e-04   
generator.value.2.weight_sigma                      4.000000e-04   

                                                                  \
                                                            mean   
layer                                                              
decoder.attn.linear_in.weight                       5.685481e-10   
decoder.attn.linear_out.weight                      6.803294e-09   
decoder.rnn.layers.0.weight_hh                      2.599324e-10   
decoder.rnn.layers.0.weight_ih                      1.330523e-09   
decoder_embeddings.make_embedding.emb_luts.0.we...  1.643254e-09   
encoder.rnn.weight_hh_l0                            4.524799e-10   
encoder.rnn.weight_hh_l0_reverse                    3.719065e-10   
encoder.rnn.weight_ih_l0                            6.571871e-09   
encoder.rnn.weight_ih_l0_reverse                    6.236895e-09   
encoder_embeddings.make_embedding.emb_luts.0.we...  3.307289e-09   
generator.advantages.0.weight_mu                    1.559721e-08   
generator.advantages.0.weight_sigma                 1.592028e-08   
generator.advantages.2.weight_mu                    2.821269e-08   
generator.advantages.2.weight_sigma                 1.807487e-08   
generator.value.0.weight_mu                         4.031903e-09   
generator.value.0.weight_sigma                      2.775448e-09   
generator.value.2.weight_mu                         5.158627e-07   
generator.value.2.weight_sigma                      3.853155e-07   

                                                                  max_grad  \
                                                          median       max   
layer                                                                        
decoder.attn.linear_in.weight                       3.508050e-12  0.000008   
decoder.attn.linear_out.weight                      8.033300e-11  0.000100   
decoder.rnn.layers.0.weight_hh                      7.397450e-12  0.000009   
decoder.rnn.layers.0.weight_ih                      2.549050e-11  0.000200   
decoder_embeddings.make_embedding.emb_luts.0.we...  2.214900e-10  0.000100   
encoder.rnn.weight_hh_l0                            9.758600e-12  0.000008   
encoder.rnn.weight_hh_l0_reverse                    5.255500e-12  0.000007   
encoder.rnn.weight_ih_l0                            5.194150e-11  0.000100   
encoder.rnn.weight_ih_l0_reverse                    6.127600e-11  0.000100   
encoder_embeddings.make_embedding.emb_luts.0.we...  8.096850e-11  0.000010   
generator.advantages.0.weight_mu                    1.986900e-10  0.000300   
generator.advantages.0.weight_sigma                 1.267950e-10  0.000400   
generator.advantages.2.weig

In [372]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Mean Gradient', range=[0, 1e-8])
fig.update_layout(title_text="Encoder Gradients")
for layer in data['layer'].unique():
    if layer.startswith('encoder'):
        fig.add_trace(go.Scatter(x=data[data['layer'] == layer]['iteration'], y=data[data['layer'] == layer]['mean_grad'],
                            mode='lines+markers', name=layer, marker=dict(
                    opacity=0.5,
                )))
fig.show()

In [373]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Mean Gradient', range=[0, 1e-8])
fig.update_layout(title_text="Decoder Gradients")
for layer in data['layer'].unique():
    if layer.startswith('decoder'):
        fig.add_trace(go.Scatter(x=data[data['layer'] == layer]['iteration'], y=data[data['layer'] == layer]['mean_grad'],
                            mode='lines+markers', name=layer, marker=dict(
                    opacity=0.5,
                )))
fig.show()

In [374]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Mean Gradient', range=[0, 1e-6])
fig.update_layout(title_text="Generator Gradients")
for layer in data['layer'].unique():
    if layer.startswith('generator') and 'mu' in  layer:
        fig.add_trace(go.Scatter(x=data[data['layer'] == layer]['iteration'], y=data[data['layer'] == layer]['mean_grad'],
                            mode='lines+markers', name=layer, marker=dict(
                    opacity=0.5,
                )))
fig.show()

# Loss

In [375]:
data =pd.read_csv('logs/td.csv', header=None)

In [376]:
data.columns = ['iteration', 'value']

In [377]:
data =data.groupby(lambda idx: (idx+1) // 3).agg({'iteration': ['max'], 'value': 'sum'})
data.columns = data.columns.droplevel(1)

In [378]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Loss')
fig.update_layout(title_text='Loss')
fig.add_trace(go.Scatter(x=data['iteration'][50:-1], y=data['value'][50:-1],
                    mode='lines+markers', name='markers', marker=dict(
            color='MediumPurple',
            opacity=0.5,
        )))

# Sigma Magnitudes

In [379]:
data =pd.read_csv('logs/sig_param_mag.csv', header=None)

In [380]:
data.columns = ['iteration', 'value']

In [381]:
fig = go.Figure()
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Mean Absolute Sigma')
fig.update_layout(title_text='Sigma')
fig.add_trace(go.Scatter(x=data['iteration'], y=data['value'],
                    mode='lines+markers', name='markers', marker=dict(
            color='MediumPurple',
            opacity=0.5,
        )))